In [6]:
import pandas as pd
import nltk
# from nltk.tokenize import word_tokenize
import string
import numpy as np
import re
import torch
import random
from torch import nn
import torch.optim as optim
import gensim
from gensim.models import Word2Vec
from torch.nn.utils.rnn import pad_sequence

In [4]:
pip install gensim


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 604.6 kB/s eta 0:00:00m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 472.5 kB/s eta 0:00:001m417.6 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 889.7 kB/s eta 0:00:001m887.0 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 1.3 MB/s eta 0:00:00MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import torch.nn.functional as F
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence

In [11]:
import contractions
from collections import Counter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/lokesh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#### contraction use

In [12]:
import contractions

text = "I can't believe it's already noon."
expanded_text = contractions.fix(text)

print(expanded_text)

I cannot believe it is already noon.


In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


---

<br>

## Task0: PREPROCES, TOKENIZE, MODEL SAVE/LOAD & LSTM module
## IMP NOTE:
- Unknown words are stored in VOCAB with <*UNK*> tag.
- Although padding is NOT done as the model is NOT trained in batches due to some issues, still the <*PAD*> tag is added to the VOCAB.

In [18]:
train_path = "./file/train.csv"
test_path = "./file/test.csv"
mainp = "./"

#### Preprocess

In [14]:
def preprocess(text, torw):
    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = contractions.fix(text)
    text = text.replace(r'\d+', ' <NUM> ')
    text = re.sub(r'#(\w+|\d+)', '<HASHTAG>', text)
    text = re.sub(r'\b(?!(i|a)\b)\w\b', '', text)
    # r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '<URL>', text)
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA> ')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', ' <HYPHENS> ')
    text = text.replace('-', ' ')
    text = text.replace('\n', ' <NEWLINE> ')
    text = text.replace(':', ' <COLON> ')
    text = text.replace('\\', ' ')
    if torw:
        words = text.split()
        return words
    else:
        return text

In [15]:
def writeTxtFile(path, X):
    with open(path, 'w') as file:
        for item in X:
            file.write("%s\n"%item)

In [20]:
df = pd.read_csv(train_path, header=None, names=['Class', 'Sentence'])
traindata = df['Sentence'].to_list()
df2 = pd.read_csv(test_path, header=None, names=['Class', 'Sentence'])
testdata = df2['Sentence'].to_list()

train_classes = df['Class'].to_list()
test_classes = df2['Class'].to_list()
trainclassfile = mainp + "class.txt"
trainfile = mainp + "train.txt"
testclassfile = mainp + "test.txt"
testfile = mainp + "testclass.txt"

writeTxtFile(trainclassfile, train_classes)
writeTxtFile(trainfile, traindata)
writeTxtFile(testfile, testdata)
writeTxtFile(testclassfile, test_classes)

In [21]:
def tokenizeWords(filename):
    with open(filename) as f:
        text = f.read()
    trash = ['/', '=', '+', '-', '*', '&', '|', '%', '^', '~', '!', '<', '>', '$', '#']
    for t in trash:
        text = text.replace(t, '')
    words = preprocess(text, True)
    pp_text = preprocess(text, False)
    punctuations = ['<PERIOD>', '<COMMA>', '<QUOTATION_MARK>', '<SEMICOLON>', '<EXCLAMATION_MARK>', '<QUESTION_MARK>',
                    '<LEFT_PAREN>', '<RIGHT_PAREN>', '<HYPHENS>', '<NEWLINE>', '<COLON>']
    sentences = pp_text.split('<NEWLINE>')
    tokenized_sentences = []
    for sentence in sentences:
        tokens = re.findall(r'\b\w+\b|<[\w\s]+>', sentence)
        tokens = [token for token in tokens if token not in punctuations]
        tokenized_sentences.append(tokens)
    words = [w for w in words if w not in punctuations]
    return words, tokenized_sentences

In [22]:
tokens,tokenized_sentences = tokenizeWords(trainfile)
vocab = Counter(tokens)
vocab['<UNK>'] = 1
vocab['<PAD>'] = 1
vocab['<bos>'] = 1
vocab['<eos>'] = 1
print(len(vocab))
sorted_vocab = sorted(vocab, key=lambda k : vocab[k], reverse=True)
indtoword = {ind:word for ind, word in enumerate(sorted_vocab)}
wordtoind = {word:ind for ind,word in indtoword.items()}
# print(indtoword)
# print(list(wordtoind.keys())[:100])
# wordtoind['<UNK>'] = 77431
# indtoword[77431] = '<UNK>'
# wordtoind['<PAD>'] = 77432
# indtoword[77432] = '<PAD>'
print(len(indtoword))
print(len(wordtoind))
print(tokens[:100])
words = list(vocab.keys())
int_words = [wordtoind[word] for word in words]
print(len(int_words))
print(int_words[:100])

77435
77435
77435
['description', 'reuters', 'shortsellers', 'wall', "street'", 'dwindling', 'band', 'of', 'ultracynics', 'are', 'seeing', 'green', 'again', 'reuters', 'private', 'investment', 'firm', 'carlyle', 'group', 'which', 'has', 'a', 'reputation', 'for', 'making', 'welltimed', 'and', 'occasionally', 'controversial', 'plays', 'in', 'the', 'defense', 'industry', 'has', 'quietly', 'placed', 'its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', 'reuters', 'soaring', 'crude', 'prices', 'plus', 'worries', 'about', 'the', 'economy', 'and', 'the', 'outlook', 'for', 'earnings', 'are', 'expected', 'to', 'hang', 'over', 'the', 'stock', 'market', 'next', 'week', 'during', 'the', 'depth', 'of', 'the', 'summer', 'doldrums', 'reuters', 'authorities', 'have', 'halted', 'oil', 'export', 'flows', 'from', 'the', 'main', 'pipeline', 'in', 'southern', 'iraq', 'after', 'intelligence', 'showed', 'a', 'rebel', 'militia', 'could', 'strike', 'infrastructure', 'an']
77435
[2974, 22, 35937, 399, 

In [23]:
print(type(tokens))
print(len(tokens))
print(len(tokenized_sentences))

<class 'list'>
3732715
120002


In [24]:
def getInputsLabels(tokenized_sentences, train_classes):
    sentence_indices = [torch.tensor([wordtoind[word] if word in wordtoind else wordtoind['<UNK>'] for word in sentence])
                    for sentence in tokenized_sentences]
    A = [[wordtoind[word] if word in wordtoind else wordtoind['<UNK>'] for word in sentence]
                    for sentence in tokenized_sentences]
 
    print(len(sentence_indices))
    # Xtrain = torch.tensor(sentence_indices, dtype=torch.long)  # Convert your list of sentence indices into a PyTorch tensor
    Xtrain = sentence_indices
    # print(train_classes)
    Ytrain = [int(y) for y in train_classes]
    Ytrain = [t - 1 for t in Ytrain]
    Ytrain = [torch.tensor(y) for y in Ytrain]
    print(len(Ytrain))
    # print(Ytrain[0])
    return Xtrain, Ytrain, A

In [25]:
Xtrain, Ytrain, Ztrain = getInputsLabels(tokenized_sentences[1:10001], train_classes[1:10001])
print(Xtrain[0])
print(Ytrain[0])
print(Ztrain[0])

10000
10000
tensor([   22, 35937,   399,   339, 10824,  3101,     3, 35938,    30,  3552,
          797,   403])
tensor(2)
[22, 35937, 399, 339, 10824, 3101, 3, 35938, 30, 3552, 797, 403]


In [26]:
test_tokens,test_sentences = tokenizeWords(testfile)
Xtest, Ytest, Ztest = getInputsLabels(test_sentences[1:2001], test_classes[1:2001])

2000
2000


### Elmo

---
---

<br>

## SAVE/LOAD models

In [39]:
def saveModel(model, path):
    torch.save(model.state_dict(), path)

def loadModel(model, path):
    if device == 'cuda':
        model.load_state_dict(torch.load(path))
    else:
        model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

---
---
<br>

## Train Word2vec

In [40]:
vocab_words = list(vocab.keys())
for i in range(len(tokenized_sentences)):
    tokenized_sentences[i] = ['<bos>'] + tokenized_sentences[i] + ['<eos>']
print(tokenized_sentences[:10])

[['<bos>', '<bos>', '<bos>', 'description', '<eos>', '<eos>', '<eos>'], ['<bos>', '<bos>', '<bos>', 'reuters', 'shortsellers', 'wall', 'street', 'dwindling', 'band', 'of', 'ultracynics', 'are', 'seeing', 'green', 'again', '<eos>', '<eos>', '<eos>'], ['<bos>', '<bos>', '<bos>', 'reuters', 'private', 'investment', 'firm', 'carlyle', 'group', 'which', 'has', 'a', 'reputation', 'for', 'making', 'welltimed', 'and', 'occasionally', 'controversial', 'plays', 'in', 'the', 'defense', 'industry', 'has', 'quietly', 'placed', 'its', 'bets', 'on', 'another', 'part', 'of', 'the', 'market', '<eos>', '<eos>', '<eos>'], ['<bos>', '<bos>', '<bos>', 'reuters', 'soaring', 'crude', 'prices', 'plus', 'worries', 'about', 'the', 'economy', 'and', 'the', 'outlook', 'for', 'earnings', 'are', 'expected', 'to', 'hang', 'over', 'the', 'stock', 'market', 'next', 'week', 'during', 'the', 'depth', 'of', 'the', 'summer', 'doldrums', '<eos>', '<eos>', '<eos>'], ['<bos>', '<bos>', '<bos>', 'reuters', 'authorities', 'hav

In [41]:
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=3, min_count=1, sg=0)

In [42]:
word2vec_model

---
---
<br>

## ELMO Architecture

In [43]:
class ELMO(nn.Module):

    def __init__(self, initial_embedding, hidden_size, num_layers):
        super().__init__()
        # Convert initial embedding to a torch tensor
        initial_embedding = torch.tensor(initial_embedding)
        r = initial_embedding.shape[0]  # Number of embeddings
        c = initial_embedding.shape[1]  # Embedding dimension
        # Initialize nn.Embedding layer with initial embeddings
        self.embedding = nn.Embedding(num_embeddings=r, embedding_dim=c, _weight=initial_embedding)
        self.embedding_dim = self.embedding.embedding_dim
        # Initialize the first bidirectional LSTM layer
        self.biLstm1 = nn.LSTM(input_size=self.embedding_dim, hidden_size=hidden_size, bidirectional=True, batch_first=True)
        # Initialize the second bidirectional LSTM layer
        self.biLstm2 = nn.LSTM(input_size=self.embedding_dim, hidden_size=hidden_size, bidirectional=True, batch_first=True)
        # Initialize learnable parameters for lambda weights
        self.lambda1 = nn.Parameter(torch.rand(1))
        self.lambda2 = nn.Parameter(torch.rand(1))
        self.lambda3 = nn.Parameter(torch.rand(1))
        # Initialize a linear layer for output projection
        self.linear = nn.Linear(in_features=self.embedding_dim, out_features=self.embedding.num_embeddings)

    def forward(self, x, train_hai_kya):
        # Convert input indices to embeddings
        x1 = self.embedding(x)
        # Pass embeddings through the first bidirectional LSTM
        x2, _ = self.biLstm1(x1)
        # Pass the output of the first LSTM through the second bidirectional LSTM
        x3, _ = self.biLstm2(x2)
        # Combine the original embeddings and LSTM outputs using the lambda weights
        output = self.lambda1 * x1 + self.lambda2 * x2 + self.lambda3 * x3
        # If training, pass the combined output through the linear layer
        if train_hai_kya:
            output = self.linear(output)
        # Return the final output
        return output


In [44]:
vocab2 = word2vec_model.wv.index_to_key
# print(vocab2)
# Iterate over each word in the vocabulary and get its embedding
word_embeddings = {}
# print(word2vec_model.wv.get_vector('the'))
for word in vocab2:
    if word in vocab:
        word_embeddings[word] = word2vec_model.wv.get_vector(word)
    else:
        word_embeddings[word] = np.random.uniform(-1, 1, 100)

In [21]:
print(word_embeddings['description'])

[-0.11873174  0.1600944  -0.3849983  -0.12125326  0.15638408 -0.33642703
  0.06397893  0.39079764 -0.19299167 -0.87419635 -0.01818905 -0.23268312
 -0.05172287  0.26084235 -0.01098538 -0.3715263   0.10706121 -0.00361197
  0.090255   -0.35577205  0.01600748 -0.09125102 -0.08357946 -0.14577334
 -0.61996853  0.06366983 -0.23688564 -0.243487    0.06257419 -0.02770707
  0.7138178  -0.26291117  0.50993204 -0.6982066   0.33807194 -0.05800671
 -0.05150339  0.01678454 -0.08136652 -0.36394235 -0.16552241 -0.48685053
  0.25285515  0.02524653  0.25189486 -0.3115349   0.05726112 -0.14774159
  0.3876161   0.17047015  0.48917517 -0.11599166  0.00804204  0.01128437
 -0.12147155  0.06933776  0.44856444  0.4166338  -0.18842885 -0.2343725
 -0.09013864  0.18499556  0.14804661  0.8547083   0.07118843  0.4500671
 -0.38063237  0.2824629  -0.20178904  0.37176758 -0.35268873  0.22064966
  0.57640195  0.39624244  0.4464895   0.14952709  0.18189196  0.62361115
  0.1145765  -0.04874411 -0.32187146 -0.05467396 -0.0

In [45]:
def shift_sentences(sentences):
    shifted_sentences = []
    for sentence in sentences:
        # shifted_sentence = torch.cat((sentence[1:], torch.tensor([wordtoind['<eos>']])))
        shifted_sentence = sentence[1:] + [wordtoind['<eos>']]
        shifted_sentences.append(shifted_sentence)

    return shifted_sentences

In [46]:
# Xtrain_shift = shift_sentences(Xtrain)
# print(len(Xtrain_shift))
Ztrain_shift = shift_sentences(Ztrain)
print(Ztrain[0])
print(Ztrain_shift[0])

[22, 35937, 399, 339, 10824, 3101, 3, 35938, 30, 3552, 797, 403]
[35937, 399, 339, 10824, 3101, 3, 35938, 30, 3552, 797, 403, 77434]


In [47]:
initial_embedding = torch.zeros(len(wordtoind), 100)
for word, index in wordtoind.items():
    if word in word_embeddings:
        initial_embedding[index] = torch.tensor(word_embeddings[word])
print(initial_embedding.shape)

torch.Size([77435, 100])


In [48]:
print(initial_embedding[22].shape)

torch.Size([100])


In [49]:
elmo_model = ELMO(initial_embedding, hidden_size = 50, num_layers = 2).to(device)

/tmp/ipykernel_27068/3043256211.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  initial_embedding = torch.tensor(initial_embedding)


In [50]:
optimizer = torch.optim.Adam(elmo_model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [52]:
def train(Xtrain, Xtrain_shift, epochs, batch_size):
    # Sabhi model parameters ko gradient update ke liye enable karna
    for p in elmo_model.parameters():
        p.requires_grad = True
    # Embedding layer ke parameters ko freeze karna taaki ve update na hoon
    for p in elmo_model.embedding.parameters():
        p.requires_grad = False

    # Lambda parameters (lambda1, lambda2, lambda3) ko freeze karna taaki ve update na hoon
    elmo_model.lambda1.requires_grad = False
    elmo_model.lambda2.requires_grad = False
    elmo_model.lambda3.requires_grad = False

    no_of_batches = len(Xtrain) // batch_size
    if len(Xtrain) % batch_size != 0:
        no_of_batches += 1
    
    for epoch in range(epochs):
        epoch_loss = 0   
        # Tqdm use karke progress bar  
        with tqdm(range(no_of_batches), desc=f"Epoch: {epoch+1}") as pbar:
            for i in range(no_of_batches):
                optimizer.zero_grad()  # Optimizer ke gradients reset karna
                batch_begin = i * batch_size
                batch_end = min((i + 1) * batch_size, len(Xtrain))
                loss = 0   
                # Current batch mein har example ke liye iterate karna
                for j in range(batch_begin, batch_end):
                    # Input aur target sequences ko tensors mein convert karna
                    x = torch.tensor(Xtrain[j], dtype=torch.long, device=device)
                    xshift = torch.tensor(Xtrain_shift[j], dtype=torch.long, device=device)
                    output = elmo_model.forward(x, True)
                    # Model output aur target ke beech loss compute karna
                    loss += criterion(output, xshift)
                epoch_loss += loss.item()  # Batch loss ko accumulate karna
                loss /= (batch_end - batch_begin)  # Batch loss ko average karna
                loss.backward()   
                optimizer.step()  # Model parameters ko update karna
                pbar.update(1)  # Progress bar ko update karna
        epoch_loss /= len(Xtrain)  # Epoch loss ka average  
        print(f"Epoch {epoch+1}, Loss: {epoch_loss}\n")   

    # Training ke baad sabhi model parameters ko freeze karna
    for parameter in elmo_model.parameters():
        parameter.requires_grad = False

    # Lambda parameters ko unfreeze karna future fine-tuning ke li


In [29]:
train(Ztrain, Ztrain_shift, 5, 32)

Epoch: 1: 100%|██████████| 313/313 [00:31<00:00,  9.94it/s]


Epoch 1, Loss: 7.711473747253418



Epoch: 2: 100%|██████████| 313/313 [00:29<00:00, 10.52it/s]


Epoch 2, Loss: 5.970930275726318



Epoch: 3: 100%|██████████| 313/313 [00:29<00:00, 10.45it/s]


Epoch 3, Loss: 5.010529235076905



Epoch: 4: 100%|██████████| 313/313 [00:30<00:00, 10.30it/s]


Epoch 4, Loss: 4.300209062194824



Epoch: 5: 100%|██████████| 313/313 [00:29<00:00, 10.51it/s]

Epoch 5, Loss: 3.717199333190918



In [31]:
saveModel(elmo_model, mainp + "elmoPT_5ep_bs32.pt")

In [40]:
print(int_words)
print(len(int_words))
print(initial_embedding[0])
print(indtoword[0])

[2974, 22, 35937, 399, 5667, 10824, 3101, 3, 35938, 30, 3552, 797, 403, 845, 715, 330, 18919, 85, 76, 18, 1, 3809, 7, 490, 35939, 5, 14555, 1255, 2687, 4, 0, 513, 221, 3417, 2031, 13, 8201, 6, 192, 288, 114, 2096, 442, 93, 1639, 1427, 58, 360, 979, 285, 204, 2, 6294, 33, 289, 95, 77, 162, 7780, 1056, 15213, 658, 27, 5039, 73, 3695, 7913, 21, 722, 3102, 451, 94, 26, 1383, 529, 968, 2517, 82, 717, 2484, 20, 264, 15, 88, 138, 45953, 52, 9193, 1694, 12903, 8370, 3438, 19, 312, 13935, 2946, 80, 240, 130, 34, 308, 522, 256, 608, 1182, 290, 49, 37, 3553, 372, 2301, 63, 11, 1788, 341, 364, 2238, 635, 10825, 1555, 146, 174, 1149, 48, 36, 1627, 1216, 897, 718, 1897, 1345, 321, 17, 668, 116, 270, 35940, 6804, 5791, 47, 2497, 46, 3056, 218, 139, 5230, 117, 2311, 1091, 804, 3810, 1557, 55, 83, 5873, 12, 2312, 20239, 4822, 3340, 8496, 35941, 45954, 4018, 45955, 9584, 1010, 353, 271, 443, 14, 1097, 646, 3170, 807, 1387, 4765, 1789, 153, 675, 317, 18920, 2378, 23, 6902, 873, 106, 45956, 751, 852, 215,

In [58]:
def get_embeddings(model, sentence):
    word_embeddings = []
    with torch.no_grad():  # Disable gradient computation
        for word_index in sentence:
            x = torch.tensor([word_index], dtype=torch.long, device=device)
            output = model.forward(x, train_hai_kya=False)  # Forward pass without training flag
            word_embeddings.append(output.cpu().numpy())  # Convert to numpy array and append to list
    return word_embeddings

In [51]:
print(Ztrain[0])

[22, 35937, 399, 339, 10824, 3101, 3, 35938, 30, 3552, 797, 403]


In [53]:
final_embedding = get_embeddings(elmo_model, Ztrain[0])

In [59]:
print(len(final_embedding))
print(final_embedding[0][0])

12
[-7.0987093e-01 -5.6622910e-01  3.2488900e-01 -4.8971605e-01
 -6.3314366e-01 -5.1151443e-01 -3.6238849e-01 -4.6255818e-01
 -5.6574291e-01 -7.9424977e-01  8.0320215e-01 -7.0647049e-01
  4.1081905e-03  5.5954343e-01 -6.6107738e-01 -2.3419607e-01
  7.5744742e-01 -6.0791302e-01  2.7292073e-02 -7.9978752e-01
 -6.4915156e-01  6.5408438e-01  7.5748265e-03 -3.4138498e-01
 -5.5021167e-01  4.4791275e-01 -3.8789761e-01  3.3360356e-01
 -6.1347950e-01  2.8866199e-01  6.3101560e-02 -4.1486341e-01
  3.7647319e-01 -5.3403163e-01 -5.9665525e-01  4.5105824e-01
 -3.0504486e-01 -7.4668515e-01 -1.9086877e-01 -1.5894163e-01
 -5.3343815e-01 -5.7919121e-01  4.8421735e-01  5.6005102e-01
  3.9607748e-01 -2.8852400e-01  5.1596254e-01 -5.9038764e-01
  5.1966977e-01 -6.2495571e-01  7.1922821e-01  4.0209472e-01
  2.6057547e-01  2.6902974e-01 -4.2432302e-01  3.7905809e-01
 -1.4736381e-01  6.4450705e-01 -3.3390820e-01  4.6261303e-02
  8.2324338e-01 -6.1037660e-01  3.6469772e-01  3.0258948e-01
 -8.4916770e-01  4.40

---
---
<br>

## Downstream Classification task

In [53]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.lstm(x)
        output = self.fc(output[:, -1, :])
        return output

In [54]:
def trainClassifier(model, elmo_model, optimizer, num_epochs, Ztrain, Ytrain):
    model.train()
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        epoch_loss = 0
        with tqdm(total=len(Ztrain), desc=f'Epoch {epoch + 1}/{num_epochs}') as pbar:
            for sent, label in zip(Ztrain, Ytrain):
                sentence_embeddings = get_embeddings(elmo_model, sent)
                optimizer.zero_grad()
                A = [torch.tensor(sentence_embeddings[i][0]) for i in range(len(sent))]
                A = torch.stack(A)
                inputs = A.unsqueeze(0)
                inputs = inputs.to(device)
                label = label.to(device)
                outputs = model(inputs)
                B = outputs.squeeze(0)
                loss = loss_fn(B, label)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                pbar.update(1)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(Ztrain)}')

In [56]:
classifier = LSTMModel(input_size=100, hidden_size=32, output_size=4).to(device)
adam2 = optim.Adam(classifier.parameters(), lr=0.001)

In [59]:
trainClassifier(classifier, elmo_model, adam2, 5, Ztrain, Ytrain)

Epoch 1/5: 100%|█████████████████████████| 10000/10000 [01:21<00:00, 122.45it/s]


Epoch [1/5], Loss: 0.565124717158731


Epoch 2/5: 100%|█████████████████████████| 10000/10000 [01:32<00:00, 108.39it/s]


Epoch [2/5], Loss: 0.4096615437910194


Epoch 3/5: 100%|█████████████████████████| 10000/10000 [01:24<00:00, 117.99it/s]


Epoch [3/5], Loss: 0.3610581576541299


Epoch 4/5: 100%|█████████████████████████| 10000/10000 [01:23<00:00, 120.01it/s]


Epoch [4/5], Loss: 0.32009243227510015


Epoch 5/5: 100%|█████████████████████████| 10000/10000 [01:22<00:00, 121.95it/s]

Epoch [5/5], Loss: 0.2788290275827574


In [60]:
saveModel(classifier, mainp + "lstm_classifier.pt")

In [64]:
from sklearn.metrics import confusion_matrix
import numpy as np

def testClassifier(model, elmo_model, Ztest, Ytest):
    model.eval()
    correct_preds = 0
    total = len(Ztest)
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for sent, label in zip(Ztest, Ytest):
            # Obtain embeddings for the sentence
            sentence_embeddings = get_embeddings(elmo_model, sent)
            # Convert embeddings to tensors
            A = [torch.tensor(sentence_embeddings[i][0]) for i in range(len(sent))]
            inputs = torch.stack(A).unsqueeze(0)
            inputs = inputs.to(device)
            label = label.to(device)
            
            # Get model outputs
            outputs = model(inputs)
            _, ypred = outputs.max(1)
            
            # Collect all true labels and predictions
            all_labels.append(label.item())
            all_preds.append(ypred.item())
            
            # Check if prediction is correct
            if ypred.item() == label.item():
                correct_preds += 1
    
    # Calculate accuracy
    acc = (correct_preds / total) * 100
    print(f'\nAccuracy on test set: {acc:.4f}%')
    
    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    print('\nConfusion Matrix:')
    print(cm)

# Example usage (ensure to have Ztest, Ytest, model, and elmo_model defined):
# testClassifier(model, elmo_model, Ztest, Ytest)


In [65]:
testClassifier(classifier, elmo_model, Ztest, Ytest)


Accuracy on test set: 86.3000%

Confusion Matrix:
[[417  37  34  23]
 [  6 508   4   8]
 [ 25   5 340  79]
 [ 18  10  25 461]]
